In [ ]:

# Long-Term Stock Recommendation and SWOT Analysis Model with Technicals and Options
!pip install yfinance pandas lxml --quiet

import yfinance as yf
import pandas as pd

PE_LOWER_BOUND = 10
PE_UPPER_BOUND = 30
DEBT_THRESHOLD = 0.5
PRICE_NEAR_52W_LOW_THRESHOLD = 1.3  # Within 30% of 52-week low

def compute_rsi(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def fetch_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        hist = stock.history(period="1y")

        hist['EMA_50'] = hist['Close'].ewm(span=50, adjust=False).mean()
        hist['EMA_200'] = hist['Close'].ewm(span=200, adjust=False).mean()
        hist['RSI'] = compute_rsi(hist['Close'])

        return {
            "Ticker": ticker,
            "Name": info.get("shortName"),
            "Sector": info.get("sector"),
            "P/E": info.get("trailingPE"),
            "Debt/Equity": info.get("debtToEquity"),
            "Current Price": info.get("currentPrice"),
            "52w Low": info.get("fiftyTwoWeekLow"),
            "52w High": info.get("fiftyTwoWeekHigh"),
            "Market Cap": info.get("marketCap"),
            "Profit Margin": info.get("profitMargins"),
            "Forward EPS": info.get("forwardEps"),
            "Revenue Growth (YoY)": info.get("revenueGrowth"),
            "Net Income": info.get("netIncomeToCommon"),
            "Dividend Yield": info.get("dividendYield"),
            "EMA_50": hist['EMA_50'].iloc[-1],
            "EMA_200": hist['EMA_200'].iloc[-1],
            "RSI": hist['RSI'].iloc[-1],
            "Options Chain": stock.option_chain(stock.options[0]) if stock.options else None
        }
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def score_stock(data):
    score = 0
    if data["Revenue Growth (YoY)"] and data["Revenue Growth (YoY)"] > 0.05: score += 1
    if data["Profit Margin"] and data["Profit Margin"] > 0.1: score += 1
    if data["RSI"] and 30 < data["RSI"] < 70: score += 1
    if data["EMA_50"] > data["EMA_200"]: score += 1
    if data["Debt/Equity"] and data["Debt/Equity"] < 0.4: score += 1
    return score

def screen_stocks(stock_list):
    results = []
    for ticker in stock_list:
        data = fetch_stock_data(ticker)
        if not data:
            continue
        if (data["P/E"] and PE_LOWER_BOUND <= data["P/E"] <= PE_UPPER_BOUND and
            data["Debt/Equity"] and data["Debt/Equity"] < DEBT_THRESHOLD and
            data["Current Price"] <= data["52w Low"] * PRICE_NEAR_52W_LOW_THRESHOLD):
            data["Score"] = score_stock(data)
            results.append(data)
    return sorted(results, key=lambda x: x["Score"], reverse=True)

# Fetch S&P 500 tickers
SP500_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
SP500_TABLE = pd.read_html(SP500_URL)
SP500_TICKERS = SP500_TABLE[0]['Symbol'].tolist()
SP500_TICKERS = [ticker.replace('.', '-') for ticker in SP500_TICKERS]

# Limit for performance (adjust if needed)
STOCK_LIST = SP500_TICKERS[:50]

# Run model
results = screen_stocks(STOCK_LIST)
pd.DataFrame(results)
